In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
print('libraries imported')

libraries imported


In [2]:
ladf = pd.read_csv("https://usc.data.socrata.com/api/views/9utn-waje/rows.csv?accessType=DOWNLOAD")
ladf=ladf.drop(['set', 'slug', 'the_geom', 'kind', 'external_i', 'display_na', 'sqmi','type','name_1','slug_1','location'], axis=1)
#the data had longitude and latitude swapped for some reason
ladf=ladf.rename(columns={"name": "Neighborhood", 'latitude':'Longitude','longitude':'Latitude'})
ladf.head(10)

,Neighborhood,Longitude,Latitude
0,Acton,-118.169810,34.497355
1,Adams-Normandie,-118.300208,34.031461
2,Agoura Hills,-118.759885,34.146736
3,Agua Dulce,-118.317104,34.504927
4,Alhambra,-118.136512,34.085539
5,Alondra Park,-118.335156,33.889617
6,Artesia,-118.080101,33.866896
7,Altadena,-118.136239,34.193871
8,Angeles Crest,-117.922395,34.313937
9,Arcadia,-118.030419,34.133230


In [7]:
ladf.shape

(272, 3)

In [3]:
#create the GET request URL
CLIENT_ID = '5OHLST3OXJSXSUTXVTVKLYBZEN1QVQRBQRJTNO25V5VW5BKN' 
CLIENT_SECRET = 'KJTZJTTNHZT1TQIN3QZESGTMCWLKHQXNCGEGOWOZQ32KXHZJ' 
VERSION = '20200525'
LIMIT=100
radius=750

In [4]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
la_venues = getNearbyVenues(names=ladf['Neighborhood'],
                                   latitudes=ladf['Latitude'],
                                   longitudes=ladf['Longitude']
                                  )
print(la_venues.shape)
la_venues.head()

Acton
Adams-Normandie
Agoura Hills
Agua Dulce
Alhambra
Alondra Park
Artesia
Altadena
Angeles Crest
Arcadia
Arleta
Arlington Heights
Athens
Atwater Village
Avalon
Avocado Heights
Azusa
Vermont-Slauson
Baldwin Hills/Crenshaw
Baldwin Park
Bel-Air
Bellflower
Bell Gardens
Green Valley
Bell
Beverly Crest
Beverly Grove
Burbank
Koreatown
Beverly Hills
Beverlywood
Boyle Heights
Bradbury
Brentwood
Broadway-Manchester
Calabasas
Canoga Park
Carson
Carthay
Castaic Canyons
Chatsworth
Castaic
Central-Alameda
Century City
Cerritos
Charter Oak
Chatsworth Reservoir
Chesterfield Square
Cheviot Hills
Chinatown
Citrus
Claremont
Northridge
Commerce
Compton
Cypress Park
La Mirada
Covina
Cudahy
Culver City
Del Aire
Del Rey
Desert View Highlands
Diamond Bar
Downey
Downtown
Duarte
Eagle Rock
East Compton
East Hollywood
East La Mirada
Elizabeth Lake
East Los Angeles
East Pasadena
East San Gabriel
Echo Park
El Monte
El Segundo
El Sereno
Elysian Park
Elysian Valley
Vermont Square
Encino
Exposition Park
Fairfax
Flo

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Acton,34.497355,-118.169810,Epik Engineering,34.498718,-118.168046,Construction & Landscaping
1,Acton,34.497355,-118.169810,Alma Gardening Co.,34.494762,-118.172550,Construction & Landscaping
2,Adams-Normandie,34.031461,-118.300208,Orange Door Sushi,34.032485,-118.299368,Sushi Restaurant
3,Adams-Normandie,34.031461,-118.300208,Shell,34.033095,-118.300025,Gas Station
4,Adams-Normandie,34.031461,-118.300208,Sushi Delight,34.032445,-118.299525,Sushi Restaurant


In [8]:
# one hot encoding
la_onehot = pd.get_dummies(la_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
la_onehot['Neighborhood'] = la_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [la_onehot.columns[-1]] + list(la_onehot.columns[:-1])
la_onehot = la_onehot[fixed_columns]

la_onehot.head()

,Yoga Studio,ATM,Accessories Store,Airport,Airport Lounge,Alternative Healer,American Restaurant,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Check Cashing Service,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Residence Hall,College Theater,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dog Run,Donburi Restaurant,Doner Restaurant,Dongbei Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Reservoir,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods 

In [9]:
#group rows by neighborhood and take mean of frequency of each occurence
la_grouped = la_onehot.groupby('Neighborhood').mean().reset_index()
la_grouped.head()

,Neighborhood,Yoga Studio,ATM,Accessories Store,Airport,Airport Lounge,Alternative Healer,American Restaurant,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Check Cashing Service,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Residence Hall,College Theater,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dog Run,Donburi Restaurant,Doner Restaurant,Dongbei Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Reservoir,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods 

In [11]:
#we just want the data on restaurants and other eateries
la_groupedrest=la_grouped[['Neighborhood','American Restaurant','Argentinian Restaurant','Asian Restaurant','Australian Restaurant','BBQ Joint','Bagel Shop','Bakery','Bar','Bistro','Brazilian Restaurant','Breakfast Spot','Brewery','Bubble Tea Shop','Buffet','Burger Joint','Burrito Place','Café','Cajun / Creole Restaurant','Cantonese Restaurant','Caribbean Restaurant','Chinese Restaurant','Coffee Shop','Cocktail Bar','Creperie','Cuban Restaurant','Cupcake Shop','Deli / Bodega','Dim Sum Restaurant','Diner','Dive Bar','Dongbei Restaurant','Donut Shop','Dumpling Restaurant','Eastern European Restaurant','English Restaurant','Ethiopian Restaurant','Falafel Restaurant','Fast Food Restaurant','Filipino Restaurant','Fish & Chips Shop','Food','Food & Drink Shop','Food Stand','Food Court','Food Truck','French Restaurant','Fried Chicken Joint','Frozen Yogurt Shop','Gastropub','Greek Restaurant','Hawaiian Restaurant','Hot Dog Joint','Ice Cream Shop','Indian Restaurant','Indonesian Restaurant','Italian Restaurant','Japanese Restaurant','Korean Restaurant','Kosher Restaurant','Latin American Restaurant','Mediterranean Restaurant','Mexican Restaurant','Middle Eastern Restaurant','New American Restaurant','Noodle House','Persian Restaurant','Peruvian Restaurant','Poke Place','Pub','Ramen Restaurant','Restaurant','Russian Restaurant','Salad Place','Sandwich Place','Satay Restaurant','Seafood Restaurant','Shabu-Shabu Restaurant','Snack Place','Soup Place','South American Restaurant','Southern / Soul Food Restaurant','Spanish Restaurant','Speakeasy','Sports Bar','Steakhouse','Sushi Restaurant','Szechuan Restaurant','Taco Place','Taiwanese Restaurant','Tea Room','Thai Restaurant','Tiki Bar','Vegetarian / Vegan Restaurant','Vietnamese Restaurant','Wine Bar']]
la_groupedrest.head()

,Neighborhood,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Bistro,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Cocktail Bar,Creperie,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food,Food & Drink Shop,Food Stand,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,Greek Restaurant,Hawaiian Restaurant,Hot Dog Joint,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Noodle House,Persian Restaurant,Peruvian Restaurant,Poke Place,Pub,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Satay Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Speakeasy,Sports Bar,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tea Room,Thai Restaurant,Tiki Bar,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Acton,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,Adams-Normandie,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.111111,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,Agoura Hills,0.037037,0.0,0.0,0.0,0.037037,0.000000,0.037037,0.0,0.0,0.0,0.074074,0.037037,0.0,0.0,0.037037,0.0,0.037037,0.0,0.0,0.0,0.074074,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.148148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.000000,0.0,0.0,0.037037,0.0,0.0,0.0,0.0
3,Agua Dulce,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,Alhambra,0.000000,0.0,0.0,0.0,0.000000,0.076923,0.000000,0.0,0.0,0.0,0.076923,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [12]:
#we want to drop the areas that have no eateries whatsoever. These areas probably aren't zoned for restaurants and will not be helpful for our analysis.
la_grouped_rest=la_groupedrest.set_index('Neighborhood')
la_grouped_rest=la_grouped_rest.loc[~(la_grouped_rest==0).all(axis=1)]
la_grouped_rest=la_grouped_rest.reset_index()
la_grouped_rest.head()

,Neighborhood,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Bistro,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Cocktail Bar,Creperie,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food,Food & Drink Shop,Food Stand,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,Greek Restaurant,Hawaiian Restaurant,Hot Dog Joint,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Noodle House,Persian Restaurant,Peruvian Restaurant,Poke Place,Pub,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Satay Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Speakeasy,Sports Bar,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tea Room,Thai Restaurant,Tiki Bar,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Adams-Normandie,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.111111,0.0,0.00,0.000000,0.0,0.0,0.0,0.0
1,Agoura Hills,0.037037,0.0,0.0,0.0,0.037037,0.000000,0.037037,0.0,0.0,0.0,0.074074,0.037037,0.0,0.0,0.037037,0.0,0.037037,0.0,0.0,0.0,0.074074,0.00,0.0,0.0,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.148148,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.000000,0.0,0.00,0.037037,0.0,0.0,0.0,0.0
2,Alhambra,0.000000,0.0,0.0,0.0,0.000000,0.076923,0.000000,0.0,0.0,0.0,0.076923,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0
3,Altadena,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0
4,Arcadia,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.050000,0.0,0.0,0.0,0.000000,0.000000,0.1,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.050000,0.05,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050000,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.050000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.05,0.050000,0.0,0.0,0.0,0.0


In [18]:
la_grouped_rest.shape

(180, 96)

In [13]:
#print each neighborhood along with top 5 most common venues
num_top_venues = 5

for hood in la_grouped_rest['Neighborhood']:
    print("----"+hood+"----")
    temp = la_grouped_rest[la_grouped_rest['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = la_grouped_rest['Neighborhood']

for ind in np.arange(la_grouped_rest.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(la_grouped_rest.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

----Adams-Normandie----
                       venue  freq
0           Sushi Restaurant  0.33
1                 Taco Place  0.11
2        American Restaurant  0.00
3  Latin American Restaurant  0.00
4                        Pub  0.00


----Agoura Hills----
                  venue  freq
0  Fast Food Restaurant  0.15
1        Breakfast Spot  0.07
2    Chinese Restaurant  0.07
3   American Restaurant  0.04
4    Mexican Restaurant  0.04


----Alhambra----
                  venue  freq
0  Fast Food Restaurant  0.08
1            Bagel Shop  0.08
2        Breakfast Spot  0.08
3    Mexican Restaurant  0.08
4   American Restaurant  0.00


----Altadena----
                      venue  freq
0          Sushi Restaurant   0.2
1       American Restaurant   0.0
2  Mediterranean Restaurant   0.0
3          Ramen Restaurant   0.0
4                       Pub   0.0


----Arcadia----
                    venue  freq
0         Bubble Tea Shop  0.10
1  Shabu-Shabu Restaurant  0.05
2     Japanese Restaurant  

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adams-Normandie,Sushi Restaurant,Taco Place,Wine Bar,Eastern European Restaurant,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar
1,Agoura Hills,Fast Food Restaurant,Chinese Restaurant,Breakfast Spot,American Restaurant,Mexican Restaurant,Restaurant,Café,Burger Joint,Deli / Bodega,Brewery
2,Alhambra,Bagel Shop,Mexican Restaurant,Fast Food Restaurant,Breakfast Spot,Wine Bar,English Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar
3,Altadena,Sushi Restaurant,Wine Bar,English Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop
4,Arcadia,Bubble Tea Shop,Coffee Shop,Chinese Restaurant,Mexican Restaurant,Fast Food Restaurant,Bakery,Shabu-Shabu Restaurant,Japanese Restaurant,Tea Room,Thai Restaurant


In [14]:
#I used the silhouette score to determine the optimal number of clusters because I found it to be the clearest metric. Scores closer to 1 are indicative of better clustering.
from sklearn.metrics import silhouette_score
U=la_grouped_rest.loc[:, la_grouped_rest.columns != 'Neighborhood']
range_n_clusters = list (range(2,14))
for n_clusters in range_n_clusters:
    clusterer = KMeans(n_clusters=n_clusters)
    preds = clusterer.fit_predict(U)
    centers = clusterer.cluster_centers_

    score = silhouette_score(U, preds)
    print("For n_clusters = {}, silhouette score is {})".format(n_clusters, score))
#the scores are not very good, but 6 appears to be the optimal number.

For n_clusters = 2, silhouette score is 0.11102235717151518)
For n_clusters = 3, silhouette score is 0.11939804976761069)
For n_clusters = 4, silhouette score is 0.11530076202227026)
For n_clusters = 5, silhouette score is 0.1362687962679363)
For n_clusters = 6, silhouette score is 0.15649880333317892)
For n_clusters = 7, silhouette score is 0.15257285133047463)
For n_clusters = 8, silhouette score is 0.16486335662565446)
For n_clusters = 9, silhouette score is 0.13825938478161462)
For n_clusters = 10, silhouette score is 0.15296662247180373)
For n_clusters = 11, silhouette score is 0.14145704359025543)
For n_clusters = 12, silhouette score is 0.14174906585603098)
For n_clusters = 13, silhouette score is 0.14435259523268124)


In [15]:
# set number of clusters
kclusters = 8

la_grouped_clustering = la_grouped_rest.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(la_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 5, 1, 1, 1, 1, 1, 1, 1, 2], dtype=int32)

In [16]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

la_merged = ladf

# add latitude/longitude for each neighborhood
la_merged = la_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='right')

la_merged.head() 

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Adams-Normandie,-118.300208,34.031461,1,Sushi Restaurant,Taco Place,Wine Bar,Eastern European Restaurant,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar
2,Agoura Hills,-118.759885,34.146736,5,Fast Food Restaurant,Chinese Restaurant,Breakfast Spot,American Restaurant,Mexican Restaurant,Restaurant,Café,Burger Joint,Deli / Bodega,Brewery
4,Alhambra,-118.136512,34.085539,1,Bagel Shop,Mexican Restaurant,Fast Food Restaurant,Breakfast Spot,Wine Bar,English Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar
6,Artesia,-118.080101,33.866896,1,Indian Restaurant,Chinese Restaurant,Vietnamese Restaurant,Bubble Tea Shop,BBQ Joint,Café,Fast Food Restaurant,Frozen Yogurt Shop,Asian Restaurant,Taiwanese Restaurant
7,Altadena,-118.136239,34.193871,1,Sushi Restaurant,Wine Bar,English Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop


In [17]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

In [19]:
# create map
map_clusters = folium.Map(location=[34.052238, -118.243340], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged['Latitude'], la_merged['Longitude'], la_merged['Neighborhood'], la_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [20]:
la_merged.loc[la_merged['Cluster Labels'] == 0, la_merged.columns[[0] + list(range(4, la_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
90,Vermont Vista,Burger Joint,Wine Bar,Creperie,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop
115,Huntington Park,Fried Chicken Joint,Burger Joint,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop
238,Valinda,Tea Room,Burger Joint,Wine Bar,English Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant


In [21]:
la_merged.loc[la_merged['Cluster Labels'] == 1, la_merged.columns[[0] + list(range(4, la_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Adams-Normandie,Sushi Restaurant,Taco Place,Wine Bar,Eastern European Restaurant,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar
4,Alhambra,Bagel Shop,Mexican Restaurant,Fast Food Restaurant,Breakfast Spot,Wine Bar,English Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar
6,Artesia,Indian Restaurant,Chinese Restaurant,Vietnamese Restaurant,Bubble Tea Shop,BBQ Joint,Café,Fast Food Restaurant,Frozen Yogurt Shop,Asian Restaurant,Taiwanese Restaurant
7,Altadena,Sushi Restaurant,Wine Bar,English Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop
9,Arcadia,Bubble Tea Shop,Coffee Shop,Chinese Restaurant,Mexican Restaurant,Fast Food Restaurant,Bakery,Shabu-Shabu Restaurant,Japanese Restaurant,Tea Room,Thai Restaurant
10,Arleta,Bakery,Wine Bar,English Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop
11,Arlington Heights,Korean Restaurant,Latin American Restaurant,Donut Shop,American Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Food,Chinese Restaurant,Bakery,Sushi Restaurant
13,Atwater Village,Taco Place,Food Court,Fried Chicken Joint,Coffee Shop,Bakery,Bar,Ice Cream Shop,Fast Food Restaurant,Mexican Restaurant,Mediterranean Restaurant
16,Azusa,Coffee Shop,Wine Bar,Creperie,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop
17,Vermont-Slauson,Chinese Restaurant,Sandwich Place,Food,Burger Joint,Wine Bar,Eastern European Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar


In [22]:
la_merged.loc[la_merged['Cluster Labels'] == 2, la_merged.columns[[0] + list(range(4, la_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Avalon,American Restaurant,Mexican Restaurant,Fried Chicken Joint,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop
15,Avocado Heights,Mexican Restaurant,Fast Food Restaurant,Diner,Taco Place,Bakery,Wine Bar,Eastern European Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant
19,Baldwin Park,Italian Restaurant,Mexican Restaurant,Wine Bar,Eastern European Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant
27,Burbank,Cuban Restaurant,Mexican Restaurant,Deli / Bodega,Sandwich Place,Bakery,Fast Food Restaurant,Dumpling Restaurant,Cupcake Shop,Dim Sum Restaurant,Diner
36,Canoga Park,Mexican Restaurant,Ice Cream Shop,Sushi Restaurant,Sports Bar,Wine Bar,Eastern European Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner
55,Cypress Park,Bakery,Mexican Restaurant,Latin American Restaurant,Wine Bar,Eastern European Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant
57,Covina,Mexican Restaurant,Burger Joint,American Restaurant,Deli / Bodega,Chinese Restaurant,Ice Cream Shop,Eastern European Restaurant,Dim Sum Restaurant,Diner,Dive Bar
68,East Compton,Mexican Restaurant,Food Truck,Food,Wine Bar,Eastern European Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar
72,East Los Angeles,Mexican Restaurant,Coffee Shop,Deli / Bodega,Fast Food Restaurant,Latin American Restaurant,Southern / Soul Food Restaurant,Wine Bar,Dumpling Restaurant,Dim Sum Restaurant,Diner
73,East Pasadena,Mexican Restaurant,Wine Bar,Fried Chicken Joint,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop


In [23]:
la_merged.loc[la_merged['Cluster Labels'] == 3, la_merged.columns[[0] + list(range(4, la_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
123,Ladera Heights,Brewery,Creperie,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop,Dumpling Restaurant
198,San Dimas,Brewery,Café,Cuban Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop,Dumpling Restaurant


In [24]:
la_merged.loc[la_merged['Cluster Labels'] == 4, la_merged.columns[[0] + list(range(4, la_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Bell,Snack Place,Wine Bar,Creperie,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop


In [25]:
la_merged.loc[la_merged['Cluster Labels'] == 5, la_merged.columns[[0] + list(range(4, la_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Agoura Hills,Fast Food Restaurant,Chinese Restaurant,Breakfast Spot,American Restaurant,Mexican Restaurant,Restaurant,Café,Burger Joint,Deli / Bodega,Brewery
22,Bell Gardens,Fried Chicken Joint,Seafood Restaurant,Fast Food Restaurant,Mexican Restaurant,Burger Joint,Latin American Restaurant,Dumpling Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner
31,Boyle Heights,Food Truck,Bakery,Donut Shop,Fast Food Restaurant,Japanese Restaurant,Café,Wine Bar,Eastern European Restaurant,Deli / Bodega,Dim Sum Restaurant
54,Compton,Fast Food Restaurant,Sandwich Place,Breakfast Spot,Burger Joint,Wine Bar,Eastern European Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar
58,Cudahy,American Restaurant,Thai Restaurant,Donut Shop,Fast Food Restaurant,Burger Joint,Eastern European Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner
60,Del Aire,Fast Food Restaurant,Breakfast Spot,Sandwich Place,Donut Shop,Sports Bar,Eastern European Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner
86,Florence,American Restaurant,Sandwich Place,Fast Food Restaurant,Mexican Restaurant,Japanese Restaurant,Dumpling Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner
93,Gramercy Park,Fast Food Restaurant,Southern / Soul Food Restaurant,Wine Bar,Eastern European Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant
95,Griffith Park,Tea Room,Fast Food Restaurant,Wine Bar,English Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant
101,Harvard Heights,Fast Food Restaurant,Korean Restaurant,Chinese Restaurant,Sandwich Place,Bakery,Donut Shop,Japanese Restaurant,Eastern European Restaurant,Deli / Bodega,Dim Sum Restaurant


In [26]:
la_merged.loc[la_merged['Cluster Labels'] == 6, la_merged.columns[[0] + list(range(4, la_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Green Valley,Food,Wine Bar,English Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop
42,Central-Alameda,Food,Donut Shop,Mexican Restaurant,Wine Bar,Eastern European Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar
81,Vermont Square,Food,Burger Joint,Wine Bar,Eastern European Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant
83,Exposition Park,Coffee Shop,Food,Wine Bar,Eastern European Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant
85,Florence-Firestone,Food,Wine Bar,English Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop
89,Green Meadows,Sandwich Place,Food Stand,Food,Donut Shop,Wine Bar,Eastern European Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner
178,Pico Rivera,Food,Wine Bar,English Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop
188,Rancho Dominguez,Bakery,Food,Wine Bar,English Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant
204,San Pasqual,American Restaurant,Food,Restaurant,Asian Restaurant,Cuban Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant
242,Val Verde,Food,Wine Bar,English Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop


In [27]:
la_merged.loc[la_merged['Cluster Labels'] == 7, la_merged.columns[[0] + list(range(4, la_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Beverly Hills,Food Truck,Breakfast Spot,Wine Bar,English Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant
33,Brentwood,Food Truck,Wine Bar,English Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop
44,Cerritos,Food Truck,Korean Restaurant,Wine Bar,English Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant
196,Rowland Heights,Food Truck,Mexican Restaurant,Wine Bar,Eastern European Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant
248,Veterans Administration,Food Truck,Wine Bar,English Restaurant,Cupcake Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Dive Bar,Dongbei Restaurant,Donut Shop
